In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00


In [5]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 23.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=6a84cc58203c54c9e004d70818e98e0078fc5c9a5acbee9aec87c35b850333ce
  Stored in directory: /home/thv200000/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [15]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.2/156.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 38.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 51.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 56.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 16.2 MB/s eta 0:00:00


In [1]:
import re

import numpy as np
from evaluate import load
from datasets import load_dataset

/home/thv200000/miniconda3/envs/code_gen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rouge = load("rouge")

In [3]:
bertscore = load("bertscore")

In [23]:
references_path = "/home/thv200000/projects/automated_code_comment/data/python_comments_test.csv"
# candidates_path = "/home/thv200000/projects/automated_code_comment/python_test_generated_comments.csv"
candidates_path = "/home/thv200000/projects/automated_code_comment/contetxt_learning_test_generated.csv"
# candidates_path = "/home/thv200000/projects/automated_code_comment/data/gen_in-context_meta-llama-Llama-2-7b-chat-hf.csv"

In [24]:
references = load_dataset('csv', data_files=references_path)
candidates = load_dataset('csv', data_files=candidates_path)

In [11]:
# cand = ["This is good.", "This is good."]
# hyp = ["This is a boy.", "This is good."]

In [12]:
print(candidates["train"]["comment"][0])

Extracts video ID from URL.


In [52]:
def extract_docstring(sample):
    """
    Extracts the docstring from the provided Python code.

    Args:
        code (str): The Python code to be analyzed.

    Returns:
        str: The extracted docstring, or an empty string if no docstring is found.
    """
    if sample["GeneratedComment"]:
        # Match triple-quoted docstrings
        match = re.search(r'"""(.*?)"""', sample["GeneratedComment"], flags=re.DOTALL)
        if match:
            sample["GeneratedComment"] = match.group(1)
            return sample

        # Match single-quoted docstrings
        match = re.search(r"''(.*?)''", sample["GeneratedComment"], flags=re.DOTALL)
        if match:
            sample["GeneratedComment"] = match.group(1)
            return sample

    sample["GeneratedComment"] = ""
    return sample

In [53]:
candidates = candidates.map(extract_docstring)

Map: 100%|██████████| 150/150 [00:00<00:00, 4840.03 examples/s]


In [25]:
def clean_extract(sample):
    if not sample['in-context-GeneratedComment']:
        sample['in-context-GeneratedComment'] = ""
    return sample


In [26]:
candidates = candidates.map(clean_extract)

Map: 100%|██████████| 150/150 [00:00<00:00, 4966.14 examples/s]


In [28]:
rouge.compute(
    predictions=candidates["train"]["in-context-GeneratedComment"],
    references=references["train"]["comment"]
)

{'rouge1': 0.25403725776035446,
 'rouge2': 0.08974507289439254,
 'rougeL': 0.201843720251484,
 'rougeLsum': 0.2360357243504188}

In [30]:
bs_score = bertscore.compute(
     lang='en',
    predictions=candidates["train"]["in-context-GeneratedComment"],
    references=references["train"]["comment"]
)

In [31]:
np.mean(bs_score['f1'])

0.8242629075050354